# Курсовой проект по курсу Введение в обработку естественного языка


# Чат-бот в Telegram

## Установка библиотек и скриптов

In [1]:
!pip install pytelegrambotapi
!pip install annoy
!pip install pymorphy2
!pip install stop_words

     |████████████████████████████████| 102 kB 5.2 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.8.2-py3-none-any.whl size=78015 sha256=1898a467e5b9b63428e3308cc63bd1b1667870fe9f41f0b59ae36a1b0c9ae024
  Stored in directory: /root/.cache/pip/wheels/05/20/79/0ec340bba3293984eac13cbd480839b754e4edffe1e517f606
Successfully built pytelegrambotapi
     |████████████████████████████████| 646 kB 5.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391662 sha256=a58eae31e774685426ba3214692a54a6ca9109160ae80f054ff2fa3a5de32528
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 7.1 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=44b8bbd5a8401137251b735fd32301a7796c6ac004f8ad449445eca6c7e76f2b
  Sto

In [2]:
# -- coding: utf-8 -

In [3]:
import telebot
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

## Подключение google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/My Drive/Colab Notebooks/115_NLP/telegram_bot/data/'

## Загрузка модели

In [6]:
modelW2V = Word2Vec.load(data_path + 'modelw2v.bin')

In [7]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
w2v_index.load(data_path + 'w2v_index.bin')

True

In [8]:
index_map = pickle.load(open(data_path + 'index_map.pkl', 'rb'))

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [11]:
def preprocess_txt(line):   
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]    
    return ' '.join(spls)

def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [12]:
token='***********************************'  # Вместо звездочек нужно вставить токен бота Telegram API
bot = telebot.TeleBot(token)

@bot.message_handler(commands=['start', 'help'])
def startCommand(message):
    bot.send_message(
        message.chat.id, 
        'Приветствую тебя user.  \n' +
        'Чем я могу тебе помочь?')
    
@bot.message_handler(content_types=['text'])
def textMessage(message):
  if message.text.lower() == "привет":
    bot.send_message(message.from_user.id, "Привет")
  elif message.text.lower() == 'я тебя люблю':
    bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAECwTNhGVJ7bMkPLwjkjGxEs7W9F6uL4QACXgIAAladvQr-LQK1VqHnEyAE')
  else:
    TEXT = message.text.lower()
    response = get_response(TEXT, w2v_index, modelW2V, index_map)
    bot.send_message(message.chat.id, response)

@bot.message_handler(content_types=['sticker'])
def sticker_id(message):
    print(message)

In [13]:
bot.polling(none_stop=True, interval=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
